# Convolutional Neural Networks (CNN) with keras
Based on [https://cambridgespark.com/content/tutorials/convolutional-neural-networks-with-keras/index.html]

In [1]:
from keras.datasets import cifar10 # subroutines for fetching the CIFAR-10 dataset
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
import numpy as np

Using TensorFlow backend.


CNN will typically have more hyperparameters than an MLP.

## Hyperparameters:
- Batch size:the number of training examples being used simultaneously during a single iteration of the gradient descent algorithm
- Number of epochs: the number of times the training algorithm will iterate over the entire training set before terminating1
- Kernel sizes in the convolutional layers
- Pooling size in the pooling layers
- Conv_depth: Number of kernels in the convolutional layers
- Dropout probability (we will apply dropout after each pooling, and after the fully connected layer)
- Hidden size: Number of neurons in the fully connected layer of the MLP.


In [2]:
batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 200 # we iterate 200 times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons

the pixel intensity values to be in the [0,1], and use a one-hot encoding for the output labels.

the sizes will be extracted from the dataset rather than hardcoded, the number of classes is inferred from the number of unique labels in the training set, and the normalisation is performed via division by the maximum value in the training set.

N.B.: we will divide the testing set by the maximum of the training set, because our algorithms are not allowed to see the testing data before the learning process is complete, and therefore we are not allowed to compute any statistics on it, other than performing transformations derived entirely from the training set.

In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data() # fetch CIFAR-10 data

num_train, height, width, depth = X_train.shape # there are 50000 training examples in CIFAR-10 
num_test = X_test.shape[0] # there are 10000 test examples in CIFAR-10
num_classes = np.unique(y_train).shape[0] # there are 10 image classes

X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')
X_train /= np.max(X_train) # Normalise data to [0, 1] range
X_test /= np.max(X_test) # Normalise data to [0, 1] range

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

170500096/170498071 [==============================] - 397s 2us/step


## Modelling
### Network will consist of four Convolution2D layers.
- A MaxPooling2D layer following after the second and the fourth convolution. 

### After the first pooling layer, we double the number of kernels (in line with the previously mentioned principle of sacrificing height and width for more depth). 

### Afterwards, the output of the second pooling layer is flattened to 1D (via the Flatten layer), and passed through two fully connected (Dense) layers. 
- ReLU activations will once again be used for all layers except the output dense layer, which will use a softmax activation (for purposes of probabilistic classification).

- Dropout layer is applied after each pooling layer, and after the first Dense layer. This is another area where Keras shines compared to other frameworks: it has an internal flag that automatically enables or disables dropout, depending on whether the model is currently used for training or testing.

### The remainder of the model specification exactly matches our previous setup for MNIST: 
- We use the cross-entropy loss function as the objective to optimise (as its derivation is more appropriate for probabilistic tasks)
- We use the Adam optimiser for gradient descent; - We report the accuracy2 of the model (as the dataset is balanced across the ten classes)
- We hold out 10% of the data for validation purposes.

In [4]:
inp = Input(shape=(height, width, depth)) # depth goes last in TensorFlow back-end (first in Theano)
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

model.fit(X_train, Y_train,                # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation
model.evaluate(X_test, Y_test, verbose=1)  # Evaluate the trained model on the test set!

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 379s 8ms/step - loss: 1.5498 - acc: 0.4314 - val_loss: 1.2225 - val_acc: 0.5650
Epoch 2/200
45000/45000 [==============================] - 375s 8ms/step - loss: 1.1393 - acc: 0.5913 - val_loss: 0.9552 - val_acc: 0.6688
Epoch 3/200
45000/45000 [==============================] - 382s 8ms/step - loss: 0.9723 - acc: 0.6558 - val_loss: 0.8486 - val_acc: 0.7048
Epoch 4/200
45000/45000 [==============================] - 384s 9ms/step - loss: 0.8651 - acc: 0.6944 - val_loss: 0.7799 - val_acc: 0.7354
Epoch 5/200
45000/45000 [==============================] - 392s 9ms/step - loss: 0.7972 - acc: 0.7182 - val_loss: 0.7354 - val_acc: 0.7444
Epoch 6/200
45000/45000 [==============================] - 396s 9ms/step - loss: 0.7388 - acc: 0.7386 - val_loss: 0.6840 - val_acc: 0.7648
Epoch 7/200
45000/45000 [==============================] - 400s 9ms/step - loss: 0.6925 - acc: 0.7540 - val_loss: 0.6

KeyboardInterrupt: 

This model achieves an accuracy of ∼78.6% on the test set; for such a difficult task (where human performance is only around 94%), and given the relative simplicity of this model, this is a respectable result. However, more sophisticated models have recently been able to get as far as 96.53%.

I appreciate that tinkering with this model might be cumbersome if you do not have a GPU in your possession. I would, however, encourage you to apply a similar model to the previously discussed MNIST dataset; you should be able to break 99.3% accuracy on its test set with little to no effort using a CNN with dropout.